# 🦤 Dodo Source

[doit](https://github.com/pydoit/doit) is a simple, yet powerful task execution tool,
written in Python. The `DoitSource` demonstrates.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements-dodo.txt

In [ ]:
import ipylab as L
import traitlets as T
from ipydatagrid import DataGrid
from ipyforcegraph import behaviors as B
from ipyforcegraph import graphs as G
from ipyforcegraph.sources.dodo import DodoSource

DODO = "🦤"

## Create a Panel

Creating a single panel in the JupyterLab `main` area next to the notebook we're working
on makes it easy to see how the application develops.

In [ ]:
app = L.JupyterFrontEnd()
panel = L.SplitPanel([], orientation="vertical", layout=dict(overflow="hidden"))
panel.title.label = DODO
app.shell.add(panel, "main", {"mode": "split-right"})

## Create the Graph

We know we want to show a graph.

In [ ]:
fg = G.ForceGraph(layout=dict(width="100%", height="100%", flex="3"))
panel.children = [fg]

## Create the `DodoSource`

A `DodoSource` needs to know its `project_root` in order to find your `dodo.py`, and
establish the right current working directory.

> While `doit` has pluggable _loaders_, `DodoSource` only supports discovering a
> `dodo.py`

In [ ]:
source = DodoSource(project_root="..")
T.dlink((source, "project_root"), (panel.title, "label"), lambda x: f"{DODO}: {x.name}")
fg.source = source

> The app should now show a big pile of `nodes`, based on the underlying
> `pandas.DataFrame`, to be explored more fully below.

## Add `Tooltip` Behaviors

Tooltips provide a quick way to inspect parts of the graph.

In [ ]:
node_tooltip = B.NodeTooltip(
    B.Nunjucks("<b>[{{ node.type }}]</b> {{ node.name }}<br/>{{ node.doc }}")
)
link_tooltip = B.LinkTooltip(
    B.Nunjucks(
        "{{ link.source.name }}<br/><b>[{{ link.type }}]</b> <br/>{{ link.target.name }}"
    )
)
fg.behaviors = [node_tooltip, link_tooltip]

> Now, hovering over each of the nodes and edges should show some data about them.

## Create some `DataGrid`s

[ipydatagrid](https://github.com/bloomberg/ipydatagrid) provides a rich `DataGrid`
class, which also speaks `pandas.DataFrame`. Since even a relatively small `doit` task
graph can be quite large, displaying the raw data in a grid provides a more readly
inspectable (and filterable) view.

In [ ]:
grid_opts = dict(
    layout=dict(height="100%", max_height="100%", min_height="100%"),
)
node_grid = DataGrid(fg.source.nodes, **grid_opts)
link_grid = DataGrid(fg.source.links, **grid_opts)
grid_panel = L.SplitPanel(
    [node_grid, link_grid],
    orientation="horizontal",
    layout=dict(overflow="hidden", min_height="400px"),
)
panel.children = [fg, grid_panel]

> The app should now contain the graph, with two grids below it.

## Add Node and Link Selection

Both `ForceGraph` and `DataGrid` support the concept of _selection_.

In [ ]:
node_grid.selection_mode = "row"
link_grid.selection_mode = "row"
node_selection = B.NodeSelection()
link_selection = B.LinkSelection()
fg.behaviors = [*fg.behaviors, node_selection, link_selection]

> Nodes, links, and grid rows can now all be selected

## Link the Selections

While the graph and grids are now selectable, they are not expressed in the same format,
and there is no relationship between them. With `traitlets.dlink`, we can create a
semi-stable, bidrectional behavior between them.

In [ ]:
def link_grid_and_graph(grid, graph_selection, source, kind):
    last_selected = set()

    def on_grid_select(*x):
        nonlocal last_selected
        if not grid.selections:
            graph_selection.selected = []
            return
        visible = grid.get_visible_data()
        selected = []
        for selection in grid.selections:
            for i in range(selection["r1"], selection["r2"] + 1):
                selected += [int(visible.iloc[i].name)]
        if last_selected != set(selected):
            last_selected = set(selected)
            graph_selection.selected = sorted(set(selected))

    grid.observe(on_grid_select, ["selections"])

    def on_graph_select(*x):
        nonlocal last_selected
        last_selected = set(graph_selection.selected)
        if not graph_selection.selected:
            grid.selections = []
            return
        visible = grid.get_visible_data()
        c2 = visible.shape[1] - 1
        selections = []
        source_kind = getattr(source, kind)
        for index in graph_selection.selected:
            sel_id = source_kind.id.iloc[index]
            r1 = int(visible[visible.id == sel_id].iloc[0].name)
            selections += [{"r1": r1, "r2": r1, "c1": 0, "c2": c2}]
        grid.selections = selections

    graph_selection.observe(on_graph_select, ["selected"])

In [ ]:
link_grid_and_graph(node_grid, node_selection, source, "nodes")
link_grid_and_graph(link_grid, link_selection, source, "links")

## Enable Running Tasks

## Watching Tasks Run